In [28]:
import pickle
import cv2
import imutils
import numpy as np
from __future__ import print_function

In [29]:
with open('model_pickle','rb') as f:
    model = pickle.load(f)

In [30]:
def testData(image):
    image = np.array(image)
    image = image.reshape((-1, 50, 50, 1))
    y = model.predict(image)
    z = y.tolist()
    x = z[0].index(max(z[0]))
    return x

In [31]:
background = None
def run_avg(image,awt):
    global background
    if background is None:
        background = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image,background,awt)
    

In [32]:
def segment(image,threshold=25):
    global background
    diff = cv2.absdiff(background.astype('uint8'),image) 
    thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)[1]
    (_, cnts, _) = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if(len(cnts)==0):
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)
    

In [33]:
def gesture(number):
    switcher={
        0:'peace',
        1:'hand',
        2:'fist',
        3:'one'
    }
    return switcher.get(number,'Invalid')

In [34]:
# weight for running average
awt = 0.5
camera = cv2.VideoCapture(0)
#region of interest coordinates
top, right, bottom, left = 150, 250, 200, 300
num_frames = 0

while(True):
    (grabbed, frame) = camera.read()
    frame = imutils.resize(frame, width=400)
# flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    clone = frame.copy()
    (height, width) = frame.shape[:2]
    roi = frame[top:bottom, right:left]
# convert the roi to grayscale and blur it
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    if num_frames < 30:
        run_avg(gray, aWeight)
    else:
# segment the hand region
        hand = segment(gray)
        if hand is not None:
            (thresholded, segmented) = hand
# draw the segmented region and display the frame
            cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
            font = cv2.FONT_HERSHEY_DUPLEX
            number = testData(thresholded)
            sign = gesture(number)
            cv2.putText(clone,sign,(right,top-15),font,1,(0,0,255))
            cv2.imshow("Thesholded", thresholded)
# draw the segmented hand
    cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 3)
    num_frames += 1
# display the frame with segmented hand
    cv2.imshow("Video Feed", clone)
# observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF
# if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break
camera.release()
cv2.destroyAllWindows()